This notebook contains an implementation of the algorithm from the main text.

A reaction network is a finite set of species $\mathcal{X}$ and a finite set of reactions $\mathcal{R}$, where each reaction consists of two non-negative integer linear combination of the species, called reactants resp. products. In the code, each species is represented by a $\texttt{symbolic variable}$ and each reaction is represented by a list containing two $\texttt{symbolic expressions}$ in the variables.

In [1]:
#Example
#WARNING: empty complex = SR(0)
X1,X2,X3 = var('X1,X2,X3')
species = [X1,X2,X3]
reactions = [[X1 + X2,X3],
             [2*X1,X2],
             [X3,SR(0)]]

Using the above representation, one can easily type in a reaction network into Sage. However, for the computations, it is more convenient to represent a reaction network as the coefficient matrix of the reactants resp. products.

In [2]:
def StandardToMatrixform(species,reactions): 
    #Input:
    #    species  ---  list of variables
    #    reactions --- list of lists containg pairs of symbolic expressions
    #Output:
    #    A --- matrix containing the coefficients of the reactants
    #    B --- matrix containing the coefficients of the products

    Atrans = []
    Btrans = []

    for j in range(len(reactions)):
        newColA = []
        newColB = []
        for i in range(len(species)):
            newColA = newColA + [reactions[j][0].coefficient(species[i],1).pyobject()] #Convert symbolic expr. to integers
            newColB = newColB + [reactions[j][1].coefficient(species[i],1).pyobject()]
        Atrans = Atrans + [newColA]
        Btrans = Btrans + [newColB]
    A = matrix(Atrans).transpose()
    B = matrix(Btrans).transpose()
    
    return A,B

In [3]:
#Example
X1,X2,X3 = var('X1,X2,X3')
species = [X1,X2,X3]
reactions = [[X1 + X2,X3],
             [2*X1,X2],
             [X3,SR(0)]]

A,B = StandardToMatrixform(species,reactions)
print(A)
print('-------')
print(B)

[1 2 0]
[1 0 0]
[0 0 1]
-------
[0 0 0]
[0 1 0]
[1 0 0]


After modifying the reaction network (e.g. removal of intermediates, or catalysts), it might be useful to display the modified network in the standard form.

In [4]:
def MatrixToStandardform(species,A,B):
    #Input:
    #    species  ---  list of variables
    #    A --- matrix containing the coefficients of the reactants
    #    B --- matrix containing the coefficients of the products
    #    reactions --- list of lists containg pairs of polynomials
    #Output:
    #    ListOfReactions --- list of lists containg pairs of polynomials
    
    ListOfReactions = []
    n = A.nrows()
    r = A.ncols()
    
    for j in range(r):
        reactant = SR(0)
        product = SR(0)
        for i in range(n):
            reactant = reactant + A[i][j]*species[i]
            product = product + B[i][j]*species[i]
        ListOfReactions = ListOfReactions + [[reactant,product]]

    return ListOfReactions

In [5]:
#Example:
X1,X2,X3 = var('X1,X2,X3')
species = [X1,X2,X3]
A = matrix([[1,2,0],[1,0,0],[0,0,1]])
B = matrix([[0,0,0],[0,1,0],[1,0,0]])

MatrixToStandardform(species,A,B)

[[X1 + X2, X3], [2*X1, X2], [X3, 0]]

# (Step 1)

The reaction network is conservative if and only if $\exists w \in \mathbb{R}^{n}$ such that
$w^{T} N = 0$ and $w > 0$.

In [6]:
def IsConservative(var1,var2):
    #Input 1:
    #    var1 --- list of variables containing the species
    #    var2 --- list of lists containg pairs of symbolic expressions representing the reactions
    #Input 2:
    #    var1 --- matrix containing the coefficients of the reactants
    #    var2 --- matrix containing the coefficients of the products
    #Output:
    #    True or False

    #If the input is a reaction network in standard form, then compute its matrixform
    if  type(var1) == list:
        A,B = StandardToMatrixform(var1,var2)
    else:
        A,B = var1,var2
        
    N = B - A 
    n = A.nrows()
    r = A.ncols()
    
    #Build the list of inequalities of the form w > 0
    IdMatrix = matrix.identity(n)
    listInEq = []
    for i in range(0,n):
        listInEq = listInEq + [ [0] + IdMatrix.row(i).list() ]

    #Build the list of equalities of the form wN = 0     
    listEq = []
    for i in range(r):
        listEq = listEq + [ [0] + N.column(i).list() ]

    P = Polyhedron(ieqs = listInEq, eqns = listEq)
    w = P.representative_point() 
    #The point is chosen to be interior as far as possible.
    #If the polyhedron is not full-dimensional, the point is in the relative interior.

    #Check w > 0         
    if min(w.list()) > 0:
        print('The reaction network is conservative.')
        return True
    else:
        print('The reaction network is not conservative.')
        return False

In [7]:
#Example 1 - a conservative network
#Hybrid histidine kinase
HK00,HKp0,HK0p,HKpp,RR,RRp = var('HK00,HKp0,HK0p,HKpp,RR,RRp')
species_HHK = [HK00,HKp0,HK0p,HKpp,RR,RRp]
reactions_HHK = [[HK00,HKp0],
                 [HKp0,HK0p],
                 [HK0p,HKpp],
                 [HK0p + RR, HK00 + RRp],
                 [HKpp + RR, HKp0 + RRp],
                 [RRp,RR]]

A,B = StandardToMatrixform(species_HHK,reactions_HHK)
IsConservative(species_HHK,reactions_HHK)
IsConservative(A,B)

#Example 2 - a non-conservative network
#Gene transcription network
X1,X2,P1,P2,X2P1,P2P2,X1P2P2 = var('X1,X2,P1,P2,X2P1,P2P2,X1P2P2')
species_GT = [X1,X2,P1,P2,X2P1,P2P2,X1P2P2]
reactions_GT = [[X1,X1 + P1],
              [X2,X2 + P2],
              [P1, SR(0)],
              [P2,SR(0)],
              [X2 + P1,X2P1],
              [X2P1,X2 + P1],
              [2*P2,P2P2],
              [P2P2,2*P2],
              [X1 + P2P2,X1P2P2],
              [X1P2P2,X1 + P2P2]]

A,B = StandardToMatrixform(species_GT,reactions_GT)
IsConservative(species_GT,reactions_GT)
IsConservative(A,B)

The reaction network is conservative.
The reaction network is conservative.
The reaction network is not conservative.
The reaction network is not conservative.


False

Besides the criterion with minimal siphons, we implement another criterion that can verify the condition (NB).

It is known that the condition (NB) is satisfied if the network after removal of intermediates and catalyst is monomolecular.

Let $C = (A \mid B ) \in \mathbb{R}^{n \times (r+r)}$.

$k \in [n]$ is called an  intermediate, if the following two conditions are satisfied

(1) for all $j \in [r + r]: c_{k j} = 0 \text{ or } (c_{k j} = 1 \text{ and } \forall i \neq k: c_{i j} = 0)$ 

(2) $\exists j,j' \in [r]: a_{k j} = 1 \text{ and } b_{k j'} = 1$

In [8]:
def FindAnIntermediate(var1,var2):
    #Input 1:
    #    var1 --- list of variables containing the species
    #    var2 --- list of lists containg pairs of symbolic expressions representing the reactions
    #Input 2:
    #    var1 --- matrix containing the coefficients of the reactants
    #    var2 --- matrix containing the coefficients of the products
    #Output:
    #    -1 if no intermediate was found
    #    index of the intermediate
    
    #If the input is a reaction network in standard form, then compute its matrixform
    if  type(var1) == list:
        A,B = StandardToMatrixform(var1,var2)
    else:
        A,B = var1,var2
    
    n = A.nrows()
    r = A.ncols()

    #Find the first intermediate
    IndexOfInterm = -1
    for k in range(n):
        IsItInterm = True
        ExistNonzeroAkj = False
        ExistNonzeroBkj = False
            
        for j in range(r):
            #Check the matrix A
            if A[k,j] == 0:
                IsItInterm = True
            elif A[k,j] == 1:
                ExistNonzeroAkj = True
                for i in range(n):
                    if i != k and A[i,j] != 0:
                        IsItInterm = False
                        break
            else:
                IsItInterm = False
            
            if not(IsItInterm):
                break
                
            #Check the matrix B
            if B[k,j] == 0:
                IsItInterm = True
            elif B[k,j] == 1:
                ExistNonzeroBkj = True
                for i in range(n):
                    if i != k and B[i,j] != 0:
                        IsItInterm = False
                        break
            else:
                IsItInterm = False
            
            if not(IsItInterm):
                break    
                 
        if IsItInterm and ExistNonzeroAkj and ExistNonzeroBkj:
            IndexOfInterm = k
            if  type(var1) == list:
                print(var1[k],'is an intermediate.')
            break      
                     
    return  IndexOfInterm

In [9]:
#Example 1
#Gene transcription network
X1,X2,P1,P2,X2P1,P2P2,X1P2P2 = var('X1,X2,P1,P2,X2P1,P2P2,X1P2P2')
species_GT = [X1,X2,P1,P2,X2P1,P2P2,X1P2P2]
reactions_GT = [[X1,X1 + P1],
              [X2,X2 + P2],
              [P1, SR(0)],
              [P2,SR(0)],
              [X2 + P1,X2P1],
              [X2P1,X2 + P1],
              [2*P2,P2P2],
              [P2P2,2*P2],
              [X1 + P2P2,X1P2P2],
              [X1P2P2,X1 + P2P2]]

A,B = StandardToMatrixform(species_GT,reactions_GT)
FindAnIntermediate(species_GT,reactions_GT)
k = FindAnIntermediate(A,B)
print(species_GT[k])


#Example 2 
#2-site
E,F,S0,S1,S2,ES0,FS1,ES1,FS2 = var('E,F,S0,S1,S2,ES0,FS1,ES1,FS2')
species_2site = [E,F,S0,S1,S2,ES0,FS1,ES1,FS2]
reactions_2site = [[S0 + E,ES0],
                   [ES0,S0 + E],
                   [ES0,S1+E],
                   [S1 + F, FS1],
                   [FS1,S1 + F],
                   [FS1, S0 + F],
                   [S1 + E, ES1],
                   [ES1, S1 + E],
                   [ES1, S2 + E],
                   [S2 + F,FS2],
                   [FS2,S2 + F],
                   [FS2,S1 + F]]


A,B = StandardToMatrixform(species_2site,reactions_2site)
FindAnIntermediate(species_2site,reactions_2site)
k = FindAnIntermediate(A,B)
print(species_2site[k])

#Example 3 - a network with no intermediate
#Hybrid histidine kinase
HK00,HKp0,HK0p,HKpp,RR,RRp = var('HK00,HKp0,HK0p,HKpp,RR,RRp')
species_HHK = [HK00,HKp0,HK0p,HKpp,RR,RRp]
reactions_HHK = [[HK00,HKp0],
                 [HKp0,HK0p],
                 [HK0p,HKpp],
                 [HK0p + RR, HK00 + RRp],
                 [HKpp + RR, HKp0 + RRp],
                 [RRp,RR]]

A,B = StandardToMatrixform(species_HHK,reactions_HHK)
FindAnIntermediate(species_HHK,reactions_HHK)
FindAnIntermediate(A,B)

X2P1 is an intermediate.
X2P1
ES0 is an intermediate.
ES0


-1

Remove an intermediate Y as follows: 

(1) remove all reactions of the original network that involve Y

(2) add a new reaction $y \rightarrow y'$ whenever $y \rightarrow Y \rightarrow y'$ with $y \neq y'$

In [10]:
def FindZeroRowOfMatrix(A):
    #Input: 
    #    A --- a matrix
    #Output: 
    #    -1 if A does not have any zero row
    #     the index of the first zero row of A
    
    k = -1
    zero = vector(zero_matrix(1, A.ncols()))
    for i in range(A.nrows()):
        if A.row(i) == zero:
            k = i
            break
    return k

In [11]:
def RemoveAnIntermediate(var1,var2,k):
    #Input 1:
    #    var1 --- list of variables containing the species
    #    var2 --- list of lists containg pairs of symbolic expressions representing the reactions
    #    k    --- index of the intermediate that has to be removed
    #Input 2:
    #    var1 --- matrix containing the coefficients of the reactants
    #    var2 --- matrix containing the coefficients of the products
    #    k    --- index of the intermediate that has to be removed
    #Output 1:
    #    list of variables containing the species of the reduced network
    #    list of lists containg pairs of polynomials containing the reactions of the reduced network
    #Output 2:
    #    matrix containing the coefficients of the reactants of the reduced network
    #    matrix containing the coefficients of the products of the reduced network  
    
    #If the input is a reaction network in standard form, then compute its matrixform
    if  type(var1) == list:
        A,B = StandardToMatrixform(var1,var2)
    else:
        A,B = var1,var2
 
    n = A.nrows()
    r = A.ncols()

    #Collect the reactions involving k
    List_k_Reactant = []
    List_k_Product = []
    
    for j in range(r):
        #Check whether in j-th reaction k is the only reactant
        IsOtherCoeffZero = True
        for i in range(n):
            if i != k and A[i][j] != 0:
                IsOtherCoeffZero = False
                break   
        if A[k][j] == 1 and IsOtherCoeffZero:
            List_k_Reactant = List_k_Reactant + [j]
        
        #Check whether in j-th reaction k is the only product
        IsOtherCoeffZero = True
        for i in range(n):
            if i != k and B[i][j] != 0:
                IsOtherCoeffZero = False
                break    
        if B[k][j] == 1 and IsOtherCoeffZero:
            List_k_Product = List_k_Product + [j]
   
    #First add the new reactions
    NumOfNewReactions = 0
    for j in List_k_Product:
        for j2 in List_k_Reactant:
            if A.column(j) != B.column(j2):
                A = A.augment(A.column(j))
                B = B.augment(B.column(j2))
                NumOfNewReactions = NumOfNewReactions + 1
   
    #Write the reactions into matrixform
    #Set the first column of the matrix to zero and later remove it
    v = vector([0..(A.nrows()-1)]) 
    A_red = matrix(v).transpose() 
    B_red = matrix(v).transpose() 
 
    #Start with the new reactions
    for jj in [0..(NumOfNewReactions - 1)]:
        A_red = A_red.augment(A.column(r+jj))
        B_red = B_red.augment(B.column(r+jj))
    #Collect the old reactions not involving k
    for j in range(r):
        IsjInList_k_Product = False
        for jj in List_k_Product:
            if j == jj:
                IsjInList_k_Product = True
        if IsjInList_k_Product:
            continue
            
        IsjInList_k_Reactant = False
        for jj in List_k_Reactant:
            if j == jj:
                IsjInList_k_Reactant = True
        if not(IsjInList_k_Product) and not(IsjInList_k_Reactant):
            A_red = A_red.augment(A.column(j))
            B_red = B_red.augment(B.column(j))
    #Remove the first column
    A_red = A_red.delete_columns([0])
    B_red = B_red.delete_columns([0])
    
    #Remove the k-th rows of A and B 
    A_red = A_red.delete_rows([k])
    B_red = B_red.delete_rows([k])
                 
    if  type(var1) == list:
        species_red = var1.copy()
        species_red.pop(k) #Remove the species with index k
        return species_red,MatrixToStandardform(species_red,A_red,B_red)
    else:
        return A_red,B_red

In [12]:
#Example 1
#Gene transcription network
X1,X2,P1,P2,X2P1,P2P2,X1P2P2 = var('X1,X2,P1,P2,X2P1,P2P2,X1P2P2')
species_GT = [X1,X2,P1,P2,X2P1,P2P2,X1P2P2]
reactions_GT = [[X1,X1 + P1],
              [X2,X2 + P2],
              [P1, SR(0)],
              [P2,SR(0)],
              [X2 + P1,X2P1],
              [X2P1,X2 + P1],
              [2*P2,P2P2],
              [P2P2,2*P2],
              [X1 + P2P2,X1P2P2],
              [X1P2P2,X1 + P2P2]]

k = FindAnIntermediate(species_GT,reactions_GT)

species_GT_red,reactions_GT_red = RemoveAnIntermediate(species_GT,reactions_GT,k)

A,B = StandardToMatrixform(species_GT,reactions_GT)
A_red,B_red = RemoveAnIntermediate(A,B,k)

A_red2,B_red2 = StandardToMatrixform(species_GT_red,reactions_GT_red)
print(A_red == A_red2,B_red == B_red2)
print(species_GT_red)
print(reactions_GT_red)

#Example 2 
#2-site
E,F,S0,S1,S2,ES0,FS1,ES1,FS2 = var('E,F,S0,S1,S2,ES0,FS1,ES1,FS2')
species_2site = [E,F,S0,S1,S2,ES0,FS1,ES1,FS2]
reactions_2site = [[S0 + E,ES0],
                   [ES0,S0 + E],
                   [ES0,S1+E],
                   [S1 + F, FS1],
                   [FS1,S1 + F],
                   [FS1, S0 + F],
                   [S1 + E, ES1],
                   [ES1, S1 + E],
                   [ES1, S2 + E],
                   [S2 + F,FS2],
                   [FS2,S2 + F],
                   [FS2,S1 + F]]

k = FindAnIntermediate(species_2site,reactions_2site)

species_2site_red,reactions_2site_red = RemoveAnIntermediate(species_2site,reactions_2site,k)

A,B = StandardToMatrixform(species_2site,reactions_2site)
A_red,B_red = RemoveAnIntermediate(A,B,k)


A_red2,B_red2 = StandardToMatrixform(species_2site_red,reactions_2site_red)
print(A_red == A_red2,B_red == B_red2)
print(species_2site_red)
print(reactions_2site_red)

X2P1 is an intermediate.
True True
[X1, X2, P1, P2, P2P2, X1P2P2]
[[X1, P1 + X1], [X2, P2 + X2], [P1, 0], [P2, 0], [2*P2, P2P2], [P2P2, 2*P2], [P2P2 + X1, X1P2P2], [X1P2P2, P2P2 + X1]]
ES0 is an intermediate.
True True
[E, F, S0, S1, S2, FS1, ES1, FS2]
[[E + S0, E + S1], [F + S1, FS1], [FS1, F + S1], [FS1, F + S0], [E + S1, ES1], [ES1, E + S1], [ES1, E + S2], [F + S2, FS2], [FS2, F + S2], [FS2, F + S1]]


In [13]:
def RemoveAllIntermediates(var1,var2):
    #Input 1:
    #    var1 --- list of variables containing the species
    #    var2 --- list of lists containg pairs of symbolic expressions representing the reactions
    #Input 2:
    #    var1 --- matrix containing the coefficients of the reactants
    #    var2 --- matrix containing the coefficients of the products
    #Output 1:
    #    list of variables containing the species of the reduced network
    #    list of lists containg pairs of polynomials containing the reactions of the reduced network
    #Output 2:
    #    matrix containing the coefficients of the reactants of the reduced network
    #    matrix containing the coefficients of the products of the reduced network 
    
    k = FindAnIntermediate(var1,var2)
    while k != -1:
        var1,var2 = RemoveAnIntermediate(var1,var2,k)
        k = FindAnIntermediate(var1,var2)
        
    return var1,var2    

In [14]:
#Example
#2-site
E,F,S0,S1,S2,ES0,FS1,ES1,FS2 = var('E,F,S0,S1,S2,ES0,FS1,ES1,FS2')
species_2site = [E,F,S0,S1,S2,ES0,FS1,ES1,FS2]
reactions_2site = [[S0 + E,ES0],
                   [ES0,S0 + E],
                   [ES0,S1+E],
                   [S1 + F, FS1],
                   [FS1,S1 + F],
                   [FS1, S0 + F],
                   [S1 + E, ES1],
                   [ES1, S1 + E],
                   [ES1, S2 + E],
                   [S2 + F,FS2],
                   [FS2,S2 + F],
                   [FS2,S1 + F]]

species_2site_red,reactions_2site_red = RemoveAllIntermediates(species_2site,reactions_2site)
print(species_2site_red)
print(reactions_2site_red)

A,B = StandardToMatrixform(species_2site,reactions_2site)
A_red,B_red = RemoveAllIntermediates(A,B)
A_red2,B_red2 = StandardToMatrixform(species_2site_red,reactions_2site_red)
print(A_red == A_red2,B_red == B_red2)

ES0 is an intermediate.
FS1 is an intermediate.
ES1 is an intermediate.
FS2 is an intermediate.
[E, F, S0, S1, S2]
[[F + S2, F + S1], [E + S1, E + S2], [F + S1, F + S0], [E + S0, E + S1]]
True True


 $k \in [n]$ is called a catalyst if for all $j \in [r]: a_{k j} = b_{k j}$ and there exists a $j_{0}: c_{k j_{0}} \neq 0$, where $C = (A \mid B ) \in \mathbb{R}^{n \times (r+r)}$.

In [15]:
def FindACatalyst(var1,var2):
    #Input 1:
    #    var1 --- list of variables containing the species
    #    var2 --- list of lists containg pairs of symbolic expressions representing the reactions
    #Input 2:
    #    var1 --- matrix containing the coefficients of the reactants
    #    var2 --- matrix containing the coefficients of the products
    #Output :
    #     -1 if no catalyst was found
    #     index of the catalyst
    
    #If the input is a reaction network in standard form, then compute its matrixform
    if  type(var1) == list:
        A,B = StandardToMatrixform(var1,var2)
    else:
        A,B = var1,var2
    
    n = A.nrows()
    r = A.ncols()
    
    IndexOfCatal = -1
 
    for k in range(n):
        IsItCatal = True
        IsZero = True
            
        for j in range(r):
            if A[k,j] != 0:
                IsZero = False
            if A[k,j] != B[k,j]:
                IsItCatal = False
                break
               
        if IsItCatal and not(IsZero):
            IndexOfCatal = k
            break
    
    if  type(var1) == list and IndexOfCatal != -1:
        print(var1[k],'is a catalyst.')        
            
    return  IndexOfCatal

In [16]:
#Example 
E,S0,S1 = var('E,S0,S1 ')
species_cat = [E,S0,S1]
reactions_cat = [[S0 + E,S1 + E],
                   [S1,S0]]

k = FindACatalyst(species_cat,reactions_cat)

A,B = StandardToMatrixform(species_cat,reactions_cat)
FindACatalyst(A,B)

E is a catalyst.


0

In [17]:
def RemoveACatalyst(var1,var2,k):
    #Input 1:
    #    var1 --- list of variables containing the species
    #    var2 --- list of lists containg pairs of symbolic expressions representing the reactions
    #    k    --- index of the catalyst that has to be removed
    #Input 2:
    #    var1 --- matrix containing the coefficients of the reactants
    #    var2 --- matrix containing the coefficients of the products
    #    k    --- index of the catalyst that has to be removed
    #Output 1:
    #    list of variables containing the species of the reduced network
    #    list of lists containg pairs of polynomials containing the reactions of the reduced network
    #Output 2:
    #    matrix containing the coefficients of the reactants of the reduced network
    #    matrix containing the coefficients of the products of the reduced network  
    
    #If the input is a reaction network in standard form, then compute its matrixform
    if  type(var1) == list:
        A,B = StandardToMatrixform(var1,var2)
    else:
        A,B = var1,var2
    
    n = A.nrows()
    r = A.ncols()
    
    for i in range(n):
        for j in range(r):
            if i == k:
                A[i,j] = 0
                B[i,j] = 0
                
    #Remove the k-th rows of A and B 
    A_red = A.delete_rows([k])
    B_red = B.delete_rows([k])
                 
    if  type(var1) == list:
        species_red = var1.copy()
        species_red.pop(k) #Remove the species with index k
        return species_red,MatrixToStandardform(species_red,A_red,B_red)
    else:
        return A_red,B_red

In [18]:
#Example 
E,S0,S1 = var('E,S0,S1 ')
species_cat = [E,S0,S1]
reactions_cat = [[S0 + E,S1 + E],
                   [S1,S0]]

k = FindACatalyst(species_cat,reactions_cat)

A,B = StandardToMatrixform(species_cat,reactions_cat)
A_red,B_red = RemoveACatalyst(A,B,k)

species_cat_red,reactions_cat_red = RemoveACatalyst(species_cat,reactions_cat,k) 
A_red2,B_red2 = StandardToMatrixform(species_cat_red,reactions_cat_red)
print(A_red == A_red2,B_red == B_red2)
reactions_cat_red
#TODO!!! Can that be a problem that we have the same reaction twice???

E is a catalyst.
True True


[[S0, S1], [S1, S0]]

In [19]:
def RemoveAllCatalysts(var1,var2):
    #Input 1:
    #    var1 --- list of variables containing the species
    #    var2 --- list of lists containg pairs of symbolic expressions representing the reactions
    #Input 2:
    #    var1 --- matrix containing the coefficients of the reactants
    #    var2 --- matrix containing the coefficients of the products
    #Output 1:
    #    list of variables containing the species of the reduced network
    #    list of lists containg pairs of polynomials containing the reactions of the reduced network
    #Output 2:
    #    matrix containing the coefficients of the reactants of the reduced network
    #    matrix containing the coefficients of the products of the reduced network 
    
    k = FindACatalyst(var1,var2)
    while k != -1:
        var1,var2 = RemoveACatalyst(var1,var2,k)
        k = FindACatalyst(var1,var2)
        
    return var1,var2    

In [20]:
#Example
#2-site
E,F,S0,S1,S2,ES0,FS1,ES1,FS2 = var('E,F,S0,S1,S2,ES0,FS1,ES1,FS2')
species_2site = [E,F,S0,S1,S2,ES0,FS1,ES1,FS2]
reactions_2site = [[S0 + E,ES0],
                   [ES0,S0 + E],
                   [ES0,S1+E],
                   [S1 + F, FS1],
                   [FS1,S1 + F],
                   [FS1, S0 + F],
                   [S1 + E, ES1],
                   [ES1, S1 + E],
                   [ES1, S2 + E],
                   [S2 + F,FS2],
                   [FS2,S2 + F],
                   [FS2,S1 + F]]

species_2site_red,reactions_2site_red = RemoveAllIntermediates(species_2site,reactions_2site)

species_2site_red2,reactions_2site_red2 = RemoveAllCatalysts(species_2site_red,reactions_2site_red)

A,B = StandardToMatrixform(species_2site_red,reactions_2site_red)
A_red,B_red = RemoveAllCatalysts(A,B)
A_red2,B_red2 = StandardToMatrixform(species_2site_red2,reactions_2site_red2)
print(A_red == A_red2,B_red == B_red2)
print(species_2site_red2)
print(reactions_2site_red2)

ES0 is an intermediate.
FS1 is an intermediate.
ES1 is an intermediate.
FS2 is an intermediate.
E is a catalyst.
F is a catalyst.
True True
[S0, S1, S2]
[[S2, S1], [S1, S2], [S1, S0], [S0, S1]]


In [21]:
def IsMonomolecular(var1,var2):
    #Input 1:
    #    var1 --- list of variables containing the species
    #    var2 --- list of lists containg pairs of symbolic expressions representing the reactions
    #Input 2:
    #    var1 --- matrix containing the coefficients of the reactants
    #    var2 --- matrix containing the coefficients of the products
    #Output :
    #     True/False
    
    #If the input is a reaction network in standard form, then compute its matrixform
    if  type(var1) == list:
        A,B = StandardToMatrixform(var1,var2)
    else:
        A,B = var1,var2

    n = A.nrows()
    r = A.ncols()
  
    IsNetworkMono = True
    for j in range(r):
        NumOfZerosA = 0
        NumOfOnesA = 0 
        
        NumOfZerosB = 0
        NumOfOnesB = 0 
        for i in range(n):
            if A[i,j] == 0:
                NumOfZerosA = NumOfZerosA + 1
            if B[i,j] == 0:
                NumOfZerosB = NumOfZerosB + 1
            if A[i,j] == 1:
                NumOfOnesA = NumOfOnesA + 1
            if B[i,j] == 1:
                NumOfOnesB = NumOfOnesB + 1
      
        if  not( (NumOfZerosA == n) or (NumOfZerosA == n-1 and NumOfOnesA == 1) ):
            IsNetworkMono = False
            break
        if  not( (NumOfZerosB == n) or (NumOfZerosB == n-1 and NumOfOnesB == 1) ):
            IsNetworkMono = False
            break    
                            
    return IsNetworkMono

In [22]:
#Example - monomolecular
E,S0,S1 = var('E,S0,S1 ')
species_cat = [E,S0,S1]
reactions_cat = [[S0,S1],
                 [S1,S0],
                 [E,S0]]

A,B = StandardToMatrixform(species_cat,reactions_cat)
print(IsMonomolecular(A,B))
print(IsMonomolecular(species_cat,reactions_cat))

#Example - not monomolecular
E,S0,S1 = var('E,S0,S1 ')
species_cat = [E,S0,S1]
reactions_cat = [[S0 + E,S1 + E],
                   [S1,S0]]

A,B = StandardToMatrixform(species_cat,reactions_cat)
print(IsMonomolecular(A,B))
print(IsMonomolecular(species_cat,reactions_cat))

True
True
False
False


The following code checks the minimal siphon criterion for the condition (NB)

In [23]:
def GetMinimalSiphons(var1,var2):
    #Input 1:
    #    var1 --- list of variables containing the species
    #    var2 --- list of lists containg pairs of symbolic expressions representing the reactions
    #Input 2:
    #    var1 --- matrix containing the coefficients of the reactants
    #    var2 --- matrix containing the coefficients of the products
    #Output 1:
    #     list of lists, each list contains variables representing a minimal siphon
    #Output 2:
    #     list of lists, each list contains indices representing a minimal siphon
    
    #If the input is a reaction network in standard form, then compute its matrixform
    if  type(var1) == list:
        A,B = StandardToMatrixform(var1,var2)
    else:
        A,B = var1,var2
                
    #Compute all siphons
    n = A.nrows()
    r = A.ncols()
    IndexSet = Set([0..n-1])
    Subsets = list(IndexSet.subsets()) #List containing the subsets
    ListOfSiphons = []
    for Z_set in Subsets:
        IsSiphon = True
        
        if Z_set.is_empty():
            continue
        Z = list(Z_set) #Convert the subset into a list
        
        for i in Z:
            IsIGood = True
            for j in range(r):
                IsJGood = False
                if B[i][j] > 0:
                    for k in Z:
                        if A[k][j] > 0:
                            IsJGood = True
                            break
                else:
                    IsJGood =True
                
                if not(IsJGood):
                    IsIGood = False
                    break
            if not(IsIGood):
                IsSiphon = False
                break
        if IsSiphon:
            ListOfSiphons = ListOfSiphons + [Set(Z)]
    
    #Collect minimal siphons
    MinimalSiphons = []
    for Z in ListOfSiphons:
        IsMinimal = True
        for S in ListOfSiphons:
            if Z != S and S.issubset(Z):
                IsMinimal = False
                break
        if IsMinimal:
            MinimalSiphons = MinimalSiphons + [list(Z)]
                 
    if  type(var1) == list:
        MinimalSiphonsSpecies = []
        for Z in MinimalSiphons:
            Z_species = []
            for i in Z:
                Z_species = Z_species + [var1[i]]
            MinimalSiphonsSpecies = MinimalSiphonsSpecies + [Z_species]   
        return MinimalSiphonsSpecies
    else:
        return MinimalSiphons

In [24]:
#Example
E,S1,S2,P,ES1,ES2 = var('E,S1,S2,P,ES1,ES2')
species_siph = [E,S1,S2,P,ES1,ES2]
reactions_siph = [[E + S1,ES1],
                   [ES1, E + S1],
                   [E + S2,ES2],
                   [ES2,E + S2],
                   [S2 + ES1, E+P],
                   [E+P,S2 + ES1],
                   [S1 + ES2, E+P],
                   [E+P,S1+ES2],
                   [S2+ES1,S1+ES2],
                   [S1+ES2,S2+ES1]]
print(GetMinimalSiphons(species_siph,reactions_siph))

A,B = StandardToMatrixform(species_siph,reactions_siph)
print(GetMinimalSiphons(A,B))

[[E, ES1, ES2], [S1, P, ES1], [S2, P, ES2]]
[[0, 4, 5], [1, 3, 4], [2, 3, 5]]


In [25]:
def ExistBoundaryEquilibria(var1,var2):
    #Input 1:
    #    var1 --- list of variables containing the species
    #    var2 --- list of lists containg pairs of symbolic expressions representing the reactions
    #Input 2:
    #    var1 --- matrix containing the coefficients of the reactants
    #    var2 --- matrix containing the coefficients of the products
    #Output:
    #     True/False
    
    #If the input is a reaction network in standard form, then compute its matrixform
    if  type(var1) == list:
        A,B = StandardToMatrixform(var1,var2)
    else:
        A,B = var1,var2

    n = A.nrows()
    r = A.ncols()
    
    #Remove the intermediates
    A,B = RemoveAllIntermediates(A,B)

    #Remove the catalysts
    A,B = RemoveAllCatalysts(A,B)

    if IsMonomolecular(A,B):
        #Build a digraph with 
        #vertices = complexes of the reaction network
        #edges = reactions
        Complexes = Set(A.columns()).list() + Set(B.columns()).list()
        G = DiGraph(len(Complexes))
        #Add the edges to G
        for j in range(A.ncols()):
            #Find the corresponding vertex to the reactant
            for t in range(len(Complexes)):
                if Complexes[t] == A.column(j):
                    ReactantVertex = t
                    break
            
            #Find the corresponding vertex to the product
            for t in range(len(Complexes)):
                if Complexes[t] == B.column(j):
                    ProductVertex = t
                    break
            G.add_edges([(ReactantVertex,ProductVertex)])       
        
        from sage.graphs.connectivity import connected_components_subgraphs
        from sage.graphs.connectivity import is_strongly_connected
        ListOfConnComp = connected_components_subgraphs(G)
        IsStronglyConn = True
        for H in ListOfConnComp :
            if not(is_strongly_connected(H)):
                IsStronglyConn = False
                break
        if IsStronglyConn:
            print('The network after removal of intermediates and catalysts is monomolecular.')
            return False

    ListOfMinimalSiphons = GetMinimalSiphons(A, B) 

    N = B - A
    n = A.nrows()
    r = A.ncols()
    IsConditionStar = True
    for Z in ListOfMinimalSiphons:
        #Build the list of inequalities of the form w > 0
        IdMatrix = matrix.identity(n)
        listInEq = []
        for i in range(0,n):
            listInEq = listInEq + [ [0] + IdMatrix.row(i).list() ]

        #Build the list of equalities of the form wN = 0     
        listEq = []
        for i in range(r):
            listEq = listEq + [ [0] + N.column(i).list() ]
            
        P = Polyhedron(ieqs = listInEq, eqns = listEq)
        w = P.representative_point() 
        #A point as a coordinate vector. The point is chosen to be interior as far as possible. 
        #If the polyhedron is not full-dimensional, the point is in the relative interior.
        if w == 0:
            IsConditionStar = False
            break
    
    if IsConditionStar:
        print('The network does not have relevant boundary steady states.')
        return False
    else:
        print('The algorithm is inconclusive.')
        
    return True #there is a possibility of boundary equilibria

In [26]:
#Example 1 - reduced network is monomolecular
E,F,S0,S1,S2,ES0,FS1,ES1,FS2 = var('E,F,S0,S1,S2,ES0,FS1,ES1,FS2')
species_2site = [E,F,S0,S1,S2,ES0,FS1,ES1,FS2]
reactions_2site = [[S0 + E,ES0],
                   [ES0,S0 + E],
                   [ES0,S1+E],
                   [S1 + F, FS1],
                   [FS1,S1 + F],
                   [FS1, S0 + F],
                   [S1 + E, ES1],
                   [ES1, S1 + E],
                   [ES1, S2 + E],
                   [S2 + F,FS2],
                   [FS2,S2 + F],
                   [FS2,S1 + F]]
ExistBoundaryEquilibria(species_2site,reactions_2site)


#Hybrid histidine kinase
HK00,HKp0,HK0p,HKpp,RR,RRp = var('HK00,HKp0,HK0p,HKpp,RR,RRp')
species_HHK = [HK00,HKp0,HK0p,HKpp,RR,RRp]
reactions_HHK = [[HK00,HKp0],
                 [HKp0,HK0p],
                 [HK0p,HKpp],
                 [HK0p + RR, HK00 + RRp],
                 [HKpp + RR, HKp0 + RRp],
                 [RRp,RR]]
ExistBoundaryEquilibria(species_HHK,reactions_HHK)

The network after removal of intermediates and catalysts is monomolecular.
The network does not have relevant boundary steady states.


False

# (Step 2)

In [27]:
def GetFluxCone(N):
    #Input: 
    #    matrix, the stoichiometric matrix of the reaction network 
    #Output: 
    #    matrix containing the extreme vectors of the flux cone
 
    r = N.ncols()
    n = N.nrows()
       
    #Build the list of inequalities of the form v >= 0
    IdMatrix = matrix.identity(r)
    listInEq = []
    for i in range(0,r):
        listInEq = listInEq + [ [0] + IdMatrix.row(i).list() ]

    #Build the list of equalities of the form Nv = 0     
    listEq = []
    for i in range(n):
        listEq = listEq + [ [0] + N.row(i).list() ]
    
    P = Polyhedron(ieqs = listInEq, eqns = listEq)
    rays = Cone(P).rays()
    E = rays.column_matrix()
    
    return E

In [28]:
#Example
N = matrix([[1,-3,1,-3,-2],[1,3,-3,1,-3]])
GetFluxCone(N)

[ 3 15 11  2]
[ 2  1  0  0]
[ 3  0  0  1]
[ 0  0  1  1]
[ 0  6  4  0]

In [29]:
def GetDeterminantOfJacobian(var1,var2,Timing = False):
    #Input 1:
    #    var1 --- list of variables containing the species
    #    var2 --- list of lists containg pairs of symbolic expressions representing the reactions
    #Input 2:
    #    var1 --- matrix containing the coefficients of the reactants
    #    var2 --- stoichiometric matrix
    #Output:
    #    symbolic expression
    
    #If the input is a reaction network in standard form, then compute its matrixform
    if  type(var1) == list:
        A,B = StandardToMatrixform(var1,var2)
        N = B - A
    else:
        A,N = var1,var2
    
    E = GetFluxCone(N)
    n = N.nrows()
    l = E.ncols()
    print('l =',l)
    

    if Timing:
        import time
        print('Computing flux cone:',time.time())
    
    if l == 0:
        return SR(0)
  
    #Compute the Jacobi matrix in convex parameters
    h = var('h', n= N.nrows(), latex_name='h') 
    L = var('L', n=l, latex_name= r'\lambda')
    Lambda = Matrix(SR,l,1,L)
    ELambda = E*Lambda
     
    Jac = N*(diagonal_matrix(ELambda.column(0)))*A.transpose()*diagonal_matrix(h) 
    
    #Modify the Jacobi matrix by W
    #Compute the kernel 
    W2 = kernel(N).matrix()
    #Make sure that W is row reduced
    W = W2.echelon_form()

    M = Jac
    d = W.nrows()
    s = n - d

    for j in range(d):
        for i in range(n):
            if W[j,i] != 0:
                ij = i
                break
            
        #Replace the ij-th row of M by the j-th row of W
        M.set_row(ij, W.row(j))
                
    q = (-1)^s*M.determinant()            
    
    return q

In [30]:
#Example
#Hybrid histidine kinase
HK00,HKp0,HK0p,HKpp,RR,RRp = var('HK00,HKp0,HK0p,HKpp,RR,RRp')
species_HHK = [HK00,HKp0,HK0p,HKpp,RR,RRp]
reactions_HHK = [[HK00,HKp0],
                 [HKp0,HK0p],
                 [HK0p,HKpp],
                 [HK0p + RR, HK00 + RRp],
                 [HKpp + RR, HKp0 + RRp],
                 [RRp,RR]]

A,B = StandardToMatrixform(species_HHK,reactions_HHK)
print(GetDeterminantOfJacobian(A,B-A))
print(GetDeterminantOfJacobian(species_HHK,reactions_HHK))

l = 2
L0^3*L1*h0*h1*h2*h4 + 3*L0^2*L1^2*h0*h1*h2*h4 + 2*L0*L1^3*h0*h1*h2*h4 + L0^2*L1^2*h0*h1*h3*h4 + L0*L1^3*h0*h1*h3*h4 - L0^3*L1*h1*h2*h3*h4 - L0^2*L1^2*h1*h2*h3*h4 + L0^3*L1*h0*h1*h2*h5 + 2*L0^2*L1^2*h0*h1*h2*h5 + L0*L1^3*h0*h1*h2*h5 + L0^3*L1*h0*h1*h3*h5 + 2*L0^2*L1^2*h0*h1*h3*h5 + L0*L1^3*h0*h1*h3*h5 + L0^3*L1*h0*h2*h3*h5 + 2*L0^2*L1^2*h0*h2*h3*h5 + L0*L1^3*h0*h2*h3*h5 + L0^3*L1*h1*h2*h3*h5 + 2*L0^2*L1^2*h1*h2*h3*h5 + L0*L1^3*h1*h2*h3*h5
l = 2
L0^3*L1*h0*h1*h2*h4 + 3*L0^2*L1^2*h0*h1*h2*h4 + 2*L0*L1^3*h0*h1*h2*h4 + L0^2*L1^2*h0*h1*h3*h4 + L0*L1^3*h0*h1*h3*h4 - L0^3*L1*h1*h2*h3*h4 - L0^2*L1^2*h1*h2*h3*h4 + L0^3*L1*h0*h1*h2*h5 + 2*L0^2*L1^2*h0*h1*h2*h5 + L0*L1^3*h0*h1*h2*h5 + L0^3*L1*h0*h1*h3*h5 + 2*L0^2*L1^2*h0*h1*h3*h5 + L0*L1^3*h0*h1*h3*h5 + L0^3*L1*h0*h2*h3*h5 + 2*L0^2*L1^2*h0*h2*h3*h5 + L0*L1^3*h0*h2*h3*h5 + L0^3*L1*h1*h2*h3*h5 + 2*L0^2*L1^2*h1*h2*h3*h5 + L0*L1^3*h1*h2*h3*h5


# (Step 3)

In [31]:
def Check_SepVector(g, GetSepVector = False):
    #Input:
    #    g --- polynomial (symbolic expression)
    #    GetSepVector --- boolean
    #Output1 (GetSepVector = False):
    #   True --- if a strict separating vector exists
    #   False --- otherwise
    #Output 2 (GetSepVector = True):
    #   vector --- if a strict separating vector exists
    #   False --- otherwise
    
    n = len(g.variables()) #number of variables  
    
    #Collect positive and negative exponents of g
    q = g.polynomial(QQ) #Convert the symbolic expresssion 'g' into a polynomial
    supp = q.exponents()
    coeff = q.coefficients()
    
    #Store the exponents as rows of matricies
    #Set the first column to zero and later remove it
    v = vector([0..(n-1)]) 
    A = matrix(v).transpose() 
    B = matrix(v).transpose()
    
    #Store the coefficients in lists
    CA = []
    CB = []
    
    for i in range(len(supp)):
        if coeff[i] > 0:
            A = A.augment(vector(supp[i]))
            CA = CA + [coeff[i]]
        else:
            B = B.augment(vector(supp[i]))
            CB = CB + [coeff[i]]
            
    A = A.delete_columns([0])
    B = B.delete_columns([0]) 
    A = A.transpose()
    B = B.transpose()

    print('# of positive coefficients:',len(CA))
    print('# of negative coefficients:',len(CB))   
    
    if len(CA) == 0:
        print('The polynomial has no positive coefficient.')
        if GetSepVector:
            return vector([0..(n-1)]) #Any vector is a strict separating vector
        else:
            return True 
    
    if len(CB) == 0:
        print('The polynomial has no negative coefficient.')
        if GetSepVector:
            return vector([0..(n-1)]) #Any vector is a strict separating vector
        else:
            return False #the preimage of the negative real line is empty
    

    ListOfIneqs = []
    #the defining inequalities of the dual polyhedon are of the form 
    #0 <= (v c)*(-a 1) for all positive exponents a 
    #0 <= (v c)*(b -1) for all negative exponents b
    for k in range(A.nrows()):
        ListOfIneqs = ListOfIneqs + [ [0] + (-A.row(k)).list() + [1]]

    for k in range(B.nrows()):
        ListOfIneqs = ListOfIneqs + [ [0] + (B.row(k)).list() + [-1]]
         
    P = Polyhedron(ieqs = ListOfIneqs)
    vc = P.representative_point() #The point is choosen to be in the interior as far as possible
    v = vector(vc[0:(len(vc)-1)]) #v is a separating vector
    c = vc[len(vc)-1] #c is the independent term of the separating hyperplane
  
    #Is v strict?    
    IsStrict = False 
    for k in range(B.nrows()): 
        v.dot_product(B.row(k))    
        if v.dot_product(B.row(k))  > c:
            IsStrict = True
            break
    if IsStrict:
        print('The support set has a strict separating hyperplane.')
    else:
        print('The support set does not have a strict separating hyperplane.')
         
    #Plot the support of 'g' together with the separating vector v, if g has two variables
    if n == 2:  
        NgA = Polyhedron(vertices=A.rows())
        NgB = Polyhedron(vertices=B.rows())
        PL0 = plot(NgA,color = 'salmon')
        PL0 += plot(NgB,color = 'skyblue')
        PL0 += point(B.rows(),color = 'blue',size = '50',zorder = '2') 
        PL0 += point(A.rows(),color = 'red',size = '50',zorder = '2') 
        
        from sage.plot.arrow import Arrow
        t = A.row(0)[0]
        if v[0]*v[1] != 0:
            PL0 += arrow2d([t,(-v[0]*t+c)/v[1]],[v[0]+t,v[1]+(-v[0]*t+c)/v[1]],color = 'black',arrowsize = 3)   
        elif v[1] != 0: #and v[0] == 0
            PL0 += arrow2d([t,c/v[1]],[t,v[1]-c/v[1]],color = 'black',arrowsize = 3)
        elif v[0] != 0: #and v[1] == 0
            PL0 += arrow2d([c/v[0],t],[v[0] - c/v[0],t],color = 'black',arrowsize = 3)
        PL0.show() 
        
    if GetSepVector and IsStrict:
        return v,c 
    else:
        return IsStrict

In [32]:
def CheckConnectivity(var1,var2,Timing = False):
    #Input 1:
    #    var1 --- list of variables containing the species
    #    var2 --- list of lists containg pairs of symbolic expressions representing the reactions
    #Input 2:
    #    var1 --- matrix containing the coefficients of the reactants
    #    var2 --- matrix containing the coefficients of the products
    #Output:
    #    True/False --- True if 
    
    #If the input is a reaction network in standard form, then compute its matrixform
    if  type(var1) == list:
        A,B = StandardToMatrixform(var1,var2)
    else:
        A,B = var1,var2
    
    n = A.nrows()
    print('n =',n)
    r = A.ncols()
    print('r =',r)

    
    if not(IsConservative(A,B)) or ExistBoundaryEquilibria(A,B):
        print('The algorithm is inconclusive.')
        return SR(0),False #might not be connected
    if Timing:
        print('Checking technical conditions:',time.time())
   
    q = GetDeterminantOfJacobian(A,B-A,Timing)
    if Timing:
        print('Computing the determinant:',time.time())
    
    if q == SR(0):
        print('The critical polynomial is equal to zero. The algorithm is inconclusive.')
        return q,False
  
  
    IsConnected = False
    ExistSepVector = Check_SepVector(q)
    if ExistSepVector:
        IsConnected = True
        print('All the conditions are satisfied. We conclude that the parameter region of multistationarity is path connected.')
    else:
        print('The algorithm is inconclusive.')
        return q, False
    
    if Timing:
        print('Finding separating hyperplane:',time.time())
  
    return q,True

# Applications

In [33]:
#Running example
X1,X2 = var('X1,X2')
species = [X1,X2]
reactions = [[X1,X2],
             [X2,X1],
             [2*X1+X2,3*X1]]

F = CheckConnectivity(species,reactions)

n = 2
r = 3
The reaction network is conservative.
The network does not have relevant boundary steady states.
l = 2
# of positive coefficients: 3
# of negative coefficients: 1
The support set has a strict separating hyperplane.
All the conditions are satisfied. We conclude that the parameter region of multistationarity is path connected.


In [34]:
#Cell cycle, (a)
C,CP,M,MP,W,WP = var('C,CP,M,MP,W,WP')
species = [C,CP,M,MP,W,WP]
reactions = [[C+MP, C + M],
             [CP+ M, C + M],
             [M + W,MP+W],
             [M+W,M+WP],
             [C,CP],
             [WP,W]]

F = CheckConnectivity(species,reactions)

n = 6
r = 6
The reaction network is conservative.
The network does not have relevant boundary steady states.
l = 3
# of positive coefficients: 5
# of negative coefficients: 1
The support set has a strict separating hyperplane.
All the conditions are satisfied. We conclude that the parameter region of multistationarity is path connected.


In [35]:
#Hybrid histidine kinase, (b)
HK00,HKp0,HK0p,HKpp,RR,RRp = var('HK00,HKp0,HK0p,HKpp,RR,RRp')
species = [HK00,HKp0,HK0p,HKpp,RR,RRp]
reactions = [[HK00,HKp0],
                 [HKp0,HK0p],
                 [HK0p,HKpp],
                 [HK0p + RR, HK00 + RRp],
                 [HKpp + RR, HKp0 + RRp],
                 [RRp,RR]]

F = CheckConnectivity(species,reactions)

n = 6
r = 6
The reaction network is conservative.
The network does not have relevant boundary steady states.
l = 2
# of positive coefficients: 17
# of negative coefficients: 2
The support set has a strict separating hyperplane.
All the conditions are satisfied. We conclude that the parameter region of multistationarity is path connected.


In [36]:
#Allosteric reciprocal enzyme regulation, (c)
S,ST,K,P,KL,PL,SKL,STP,L = var('S,ST,K,P,KL,PL,SKL,STP,L')
species = [S,ST,K,P,KL,PL,SKL,STP,L]
reactions = [[S+KL,SKL],
             [SKL,S+KL],
             [SKL,ST+KL],
             [ST+P,STP],
             [STP,ST+P],
             [STP,S+P],
             [K+L,KL],
             [KL,K+L],
             [P+L,PL],
             [PL,P+L]]

F = CheckConnectivity(species,reactions)

n = 9
r = 10
The reaction network is conservative.
The network does not have relevant boundary steady states.
l = 5
# of positive coefficients: 168
# of negative coefficients: 8
The support set does not have a strict separating hyperplane.
The algorithm is inconclusive.


In [37]:
#Allosteric reciprocal enzyme regulation, (c) - reduced
S,ST,K,P,KL,PL,SKL,STP,L = var('S,ST,K,P,KL,PL,SKL,STP,L')
species = [S,ST,K,P,KL,PL,SKL,STP,L]
reactions = [[S+KL,SKL],
             [SKL,ST+KL],
             [ST+P,STP],
             [STP,S+P],
             [K+L,KL],
             [KL,K+L],
             [P+L,PL],
             [PL,P+L]]

F = CheckConnectivity(species,reactions)

n = 9
r = 8
The reaction network is conservative.
The network does not have relevant boundary steady states.
l = 3
# of positive coefficients: 42
# of negative coefficients: 2
The support set does not have a strict separating hyperplane.
The algorithm is inconclusive.


In [38]:
#Covalent reciprocal enzyme regulation, (d)
S,K,SK,ST,PT,STPT,K,E1,KE1,KT,P,PE1  = var('S,K,SK,ST,PT,STPT,K,E1,KE1,KT,P,PE1 ')
species = [S,K,SK,ST,PT,STPT,K,E1,KE1,KT,P,PE1 ]
reactions = [[S+K,SK],
             [SK,S+K],
             [SK,ST+K],
             
             
             [ST+PT,STPT],
             [STPT,ST+PT],
             [STPT,S+PT],
             
             [K + E1,KE1],
             [KE1,K + E1],
             [KE1,KT + E1],
             
             [P + E1,PE1],
             [PE1,P + E1],
             [PE1,PT + E1],
             
             [PT,P],
             [KT,K]]

F = CheckConnectivity(species,reactions)

n = 12
r = 14
The reaction network is conservative.
The network does not have relevant boundary steady states.
l = 7
# of positive coefficients: 1856
# of negative coefficients: 32
The support set has a strict separating hyperplane.
All the conditions are satisfied. We conclude that the parameter region of multistationarity is path connected.


In [39]:
#Covalent reciprocal enzyme regulation, (d) - reduced 
S,K,SK,ST,PT,STPT,K,E1,KE1,KT,P,PE1  = var('S,K,SK,ST,PT,STPT,K,E1,KE1,KT,P,PE1 ')
species = [S,K,SK,ST,PT,STPT,K,E1,KE1,KT,P,PE1 ]
reactions = [[S+K,SK],
             [SK,ST+K],
             
             
             [ST+PT,STPT],
             [STPT,S+PT],
             
             [K + E1,KE1],
             [KE1,KT + E1],
             
             [P + E1,PE1],
             [PE1,PT + E1],
             
             [PT,P],
             [KT,K]]

F = CheckConnectivity(species,reactions)

n = 12
r = 10
The reaction network is conservative.
The network does not have relevant boundary steady states.
l = 3
# of positive coefficients: 116
# of negative coefficients: 2
The support set has a strict separating hyperplane.
All the conditions are satisfied. We conclude that the parameter region of multistationarity is path connected.


In [40]:
#2-site phosphorylation cycle, (e)
E,F,S0,S1,S2,ES0,FS1,ES1,FS2 = var('E,F,S0,S1,S2,ES0,FS1,ES1,FS2')
species = [E,F,S0,S1,S2,ES0,FS1,ES1,FS2]
reactions = [[S0 + E,ES0],
                   [ES0,S0 + E],
                   [ES0,S1+E],
                   [S1 + F, FS1],
                   [FS1,S1 + F],
                   [FS1, S0 + F],
                   [S1 + E, ES1],
                   [ES1, S1 + E],
                   [ES1, S2 + E],
                   [S2 + F,FS2],
                   [FS2,S2 + F],
                   [FS2,S1 + F]]

import time

Start_Time = time.time()
print('Start time: ' + str(Start_Time))
F = CheckConnectivity(species,reactions,Timing = True)

Execution_Time = time.time() - Start_Time
print('Total time: '+str(Execution_Time))

Start time: 1663778250.85085
n = 9
r = 12
The reaction network is conservative.
The network after removal of intermediates and catalysts is monomolecular.
Checking technical conditions: 1663778250.9081478
l = 6
Computing flux cone: 1663778250.935473
Computing the determinant: 1663778253.2714295
# of positive coefficients: 288
# of negative coefficients: 112
The support set has a strict separating hyperplane.
All the conditions are satisfied. We conclude that the parameter region of multistationarity is path connected.
Finding separating hyperplane: 1663778254.4814053
Total time: 3.6308984756469727


In [41]:
#2-site phosphorylation cycle, (e) - reduced
E,F,S0,S1,S2,ES0,FS1,ES1,FS2 = var('E,F,S0,S1,S2,ES0,FS1,ES1,FS2')
species = [E,F,S0,S1,S2,ES0,FS1,ES1,FS2]
reactions = [[S0 + E,ES0],
             [ES0,S1+E],
             [S1 + F, FS1],
             [FS1, S0 + F],
             [S1 + E, ES1],
             [ES1, S2 + E],
             [S2 + F,FS2],
             [FS2,S1 + F] ]
import time

Start_Time = time.time()
print('Start time: ' + str(Start_Time))
F = CheckConnectivity(species,reactions,Timing = True)

Execution_Time = time.time() - Start_Time
print('Total time: '+str(Execution_Time))

Start time: 1663778254.5025668
n = 9
r = 8
The reaction network is conservative.
The network after removal of intermediates and catalysts is monomolecular.
Checking technical conditions: 1663778254.5322418
l = 2
Computing flux cone: 1663778254.5386977
Computing the determinant: 1663778255.0185335
# of positive coefficients: 18
# of negative coefficients: 7
The support set has a strict separating hyperplane.
All the conditions are satisfied. We conclude that the parameter region of multistationarity is path connected.
Finding separating hyperplane: 1663778255.1025035
Total time: 0.6002669334411621


In [42]:
#3-site phosphorylation cycle
E,F,S0,S1,S2,S3,ES0,FS1,ES1,FS2,ES2,FS3 = var('E,F,S0,S1,S2,S3,ES0,FS1,ES1,FS2,ES2,FS3')
species = [E,F,S0,S1,S2,S3,ES0,FS1,ES1,FS2,ES2,FS3]
reactions = [[S0 + E,ES0],
                   [ES0,S0 + E],
                   [ES0,S1+E],
                   [S1 + F, FS1],
                   [FS1,S1 + F],
                   [FS1, S0 + F],
                   [S1 + E, ES1],
                   [ES1, S1 + E],
                   [ES1, S2 + E],
                   [S2 + F,FS2],
                   [FS2,S2 + F],
                   [FS2,S1 + F],
                   [S2+E,ES2],
                   [ES2,S2+E],
                   [ES2,S3 + E],
                   [S3 + F,FS3],
                   [FS3, S3 + F],
                   [FS3,S2 + F]]

import time

Start_Time = time.time()
print('Start time: ' + str(Start_Time))
F = CheckConnectivity(species,reactions,Timing = True)

Execution_Time = time.time() - Start_Time
print('Total time: '+str(Execution_Time))

Start time: 1663778255.143374
n = 12
r = 18
The reaction network is conservative.
The network after removal of intermediates and catalysts is monomolecular.
Checking technical conditions: 1663778255.2422628
l = 9
Computing flux cone: 1663778255.2780771
Computing the determinant: 1663778461.9836543
# of positive coefficients: 2560
# of negative coefficients: 1536
The support set has a strict separating hyperplane.
All the conditions are satisfied. We conclude that the parameter region of multistationarity is path connected.
Finding separating hyperplane: 1663778482.2718072
Total time: 227.12882351875305


In [43]:
#3-site phosphorylation cycle -reduced
E,F,S0,S1,S2,S3,ES0,FS1,ES1,FS2,ES2,FS3 = var('E,F,S0,S1,S2,S3,ES0,FS1,ES1,FS2,ES2,FS3')
species = [E,F,S0,S1,S2,S3,ES0,FS1,ES1,FS2,ES2,FS3]
reactions = [[S0 + E,ES0],
                   [ES0,S1+E],
                   [S1 + F, FS1],
                   [FS1, S0 + F],
                   [S1 + E, ES1],
                   [ES1, S2 + E],
                   [S2 + F,FS2],
                   [FS2,S1 + F],
                   [S2+E,ES2],
                   [ES2,S3 + E],
                   [S3 + F,FS3],
                   [FS3,S2 + F]]

import time

Start_Time = time.time()
print('Start time: ' + str(Start_Time))
F = CheckConnectivity(species,reactions,Timing = True)

Execution_Time = time.time() - Start_Time
print('Total time: '+str(Execution_Time))

Start time: 1663778482.3372543
n = 12
r = 12
The reaction network is conservative.
The network after removal of intermediates and catalysts is monomolecular.
Checking technical conditions: 1663778482.525461
l = 3
Computing flux cone: 1663778482.5468063
Computing the determinant: 1663778488.201931
# of positive coefficients: 40
# of negative coefficients: 24
The support set has a strict separating hyperplane.
All the conditions are satisfied. We conclude that the parameter region of multistationarity is path connected.
Finding separating hyperplane: 1663778488.4616213
Total time: 6.124727249145508


In [44]:
#4-site phosphorylation cycle - reduced
E,F,S0,S1,S2,S3,S4,ES0,FS1,ES1,FS2,ES2,FS3,ES3,FS4 = var('E,F,S0,S1,S2,S3,S4,ES0,FS1,ES1,FS2,ES2,FS3,ES3,FS4')
species = [E,F,S0,S1,S2,S3,S4,ES0,FS1,ES1,FS2,ES2,FS3,ES3,FS4]
reactions = [[S0 + E,ES0],
             #[ES0,S0 + E],
             [ES0,S1+E],
             [S1 + F, FS1],
             #[FS1,S1 + F],
             [FS1, S0 + F],
             [S1 + E, ES1],
             #[ES1, S1 + E],
             [ES1, S2 + E],
             [S2 + F,FS2],
             #[FS2,S2 + F],
             [FS2,S1 + F],
             [S2+E,ES2],
             #[ES2,S2+E],
             [ES2,S3 + E],
             [S3 + F,FS3],
             #[FS3, S3 + F],
             [FS3,S2 + F],
             [S3 + E, ES3],
             #[ES3,S3 + E],
             [ES3, S4 + E],
             [S4 + F,FS4],
             #[FS4, S4 + F],
             [FS4, S3 + F] ]
import time

Start_Time = time.time()
print('Start time: ' + str(Start_Time))
F = CheckConnectivity(species,reactions,Timing = True)

Execution_Time = time.time() - Start_Time
print('Total time: '+str(Execution_Time))

Start time: 1663778488.4900708
n = 15
r = 16
The reaction network is conservative.
The network after removal of intermediates and catalysts is monomolecular.
Checking technical conditions: 1663778488.6203392
l = 4
Computing flux cone: 1663778488.649555
Computing the determinant: 1663778594.9160993
# of positive coefficients: 75
# of negative coefficients: 54
The support set does not have a strict separating hyperplane.
The algorithm is inconclusive.
Total time: 107.23366284370422


In [45]:
#2-site: diff. phosphatases, (f)
S0,S1,S2,E,F1,F2,S0E,S1E,S1F1,S2F2 = var('S0,S1,S2,E,F1,F2,S0E,S1E,S1F1,S2F2')
species = [S0,S1,S2,E,F1,F2,S0E,S1E,S1F1,S2F2]
reactions = [[S0 + E,S0E],
              [S0E, S1+ E],
              [S0E, S0 +E],
              [S1 + E,S1E],
              [S1E,S1 + E],
              [S1E,S2 + E],
              [S1 + F1,S1F1],
              [S1F1,S1 + F1],
              [S1F1,S0 + F1],
              [S2 + F2,S2F2],
              [S2F2,S2 + F2],
              [S2F2,S1+F2]]

F = CheckConnectivity(species,reactions)

n = 10
r = 12
The reaction network is conservative.
The network after removal of intermediates and catalysts is monomolecular.
l = 6
# of positive coefficients: 304
# of negative coefficients: 48
The support set has a strict separating hyperplane.
All the conditions are satisfied. We conclude that the parameter region of multistationarity is path connected.


In [46]:
#2-site: diff. phosphatases, (f) - reduced
S0,S1,S2,E,F1,F2,S0E,S1E,S1F1,S2F2 = var('S0,S1,S2,E,F1,F2,S0E,S1E,S1F1,S2F2')
species = [S0,S1,S2,E,F1,F2,S0E,S1E,S1F1,S2F2]
reactions = [[S0 + E,S0E],
              [S0E, S1+ E],
              #[S0E, S0 +E],
              [S1 + E,S1E],
              #[S1E,S1 + E],
              [S1E,S2 + E],
              [S1 + F1,S1F1],
              #[S1F1,S1 + F1],
              [S1F1,S0 + F1],
              [S2 + F2,S2F2],
              #[S2F2,S2 + F2],
              [S2F2,S1+F2]]

F = CheckConnectivity(species,reactions)

n = 10
r = 8
The reaction network is conservative.
The network after removal of intermediates and catalysts is monomolecular.
l = 2
# of positive coefficients: 19
# of negative coefficients: 3
The support set has a strict separating hyperplane.
All the conditions are satisfied. We conclude that the parameter region of multistationarity is path connected.


In [47]:
#3-site: diff. phosphatases
S0,S1,S2,S3,E,F1,F2,F3,S0E,S1E,S2E,S1F1,S2F2,S3F3 = var('S0,S1,S2,S3,E,F1,F2,F3,S0E,S1E,S2E,S1F1,S2F2,S3F3')
species = [S0,S1,S2,S3,E,F1,F2,F3,S0E,S1E,S2E,S1F1,S2F2,S3F3]
reactions = [[S0 + E,S0E],
              [S0E, S1+ E],
              [S0E, S0 +E],
              [S1 + E,S1E],
              [S1E,S1 + E],
              [S1E,S2 + E],
              [S2 + E,S2E],
              [S2E,S2 + E],
              [S2E,S3 + E],
              #--------------------------
              [S1 + F1,S1F1],
              [S1F1,S1 + F1],
              [S1F1,S0 + F1],
              [S2 + F2,S2F2],
              [S2F2,S2 + F2],
              [S2F2,S1+F2],
              [S3+F3,S3F3],
              [S3F3,S3+F3],
              [S3F3,S2+F3]]

F = CheckConnectivity(species,reactions)

n = 14
r = 18
The reaction network is conservative.
The network after removal of intermediates and catalysts is monomolecular.
l = 9
# of positive coefficients: 3264
# of negative coefficients: 960
The support set has a strict separating hyperplane.
All the conditions are satisfied. We conclude that the parameter region of multistationarity is path connected.


In [48]:
#3-site: diff. phosphatases - reduced
S0,S1,S2,S3,E,F1,F2,F3,S0E,S1E,S2E,S1F1,S2F2,S3F3 = var('S0,S1,S2,S3,E,F1,F2,F3,S0E,S1E,S2E,S1F1,S2F2,S3F3')
species = [S0,S1,S2,S3,E,F1,F2,F3,S0E,S1E,S2E,S1F1,S2F2,S3F3]
reactions = [[S0 + E,S0E],
              [S0E, S1+ E],
              #[S0E, S0 +E],
              [S1 + E,S1E],
              #[S1E,S1 + E],
              [S1E,S2 + E],
              [S2 + E,S2E],
              #[S2E,S2 + E],
              [S2E,S3 + E],
              #--------------------------
              [S1 + F1,S1F1],
              #[S1F1,S1 + F1],
              [S1F1,S0 + F1],
              [S2 + F2,S2F2],
              #[S2F2,S2 + F2],
              [S2F2,S1+F2],
              [S3+F3,S3F3],
              #[S3F3,S3+F3],
              [S3F3,S2+F3]]

F = CheckConnectivity(species,reactions)

n = 14
r = 12
The reaction network is conservative.
The network after removal of intermediates and catalysts is monomolecular.
l = 3
# of positive coefficients: 51
# of negative coefficients: 15
The support set has a strict separating hyperplane.
All the conditions are satisfied. We conclude that the parameter region of multistationarity is path connected.


In [49]:
#4-site: diff. phosphatases - reduced
S0,S1,S2,S3,S4,E,F1,F2,F3,F4,S0E,S1E,S2E,S3E,S1F1,S2F2,S3F3,S4F4 = var('S0,S1,S2,S3,S4,E,F1,F2,F3,F4,S0E,S1E,S2E,S3E,S1F1,S2F2,S3F3,S4F4')
species = [S0,S1,S2,S3,S4,E,F1,F2,F3,F4,S0E,S1E,S2E,S3E,S1F1,S2F2,S3F3,S4F4]
reactions = [[S0 + E,S0E],
              [S0E, S1+ E],
              #[S0E, S0 +E],
              [S1 + E,S1E],
              #[S1E,S1 + E],
              [S1E,S2 + E],
              [S2 + E,S2E],
              #[S2E,S2 + E],
              [S2E,S3 + E],
              [S3 + E,S3E],
              #[S3E,S3 + E],
              [S3E,S4 + E],
              #--------------------------
              [S1 + F1,S1F1],
              #[S1F1,S1 + F1],
              [S1F1,S0 + F1],
              [S2 + F2,S2F2],
              #[S2F2,S2 + F2],
              [S2F2,S1+F2],
              [S3+F3,S3F3],
              #[S3F3,S3+F3],
              [S3F3,S2+F3],
              [S4+F4,S4F4],
              #[S4F4,S4+F4],
              [S4F4,S3+F4]]

F = CheckConnectivity(species,reactions)

n = 18
r = 16
The reaction network is conservative.
The network after removal of intermediates and catalysts is monomolecular.
l = 4
# of positive coefficients: 127
# of negative coefficients: 51
The support set has a strict separating hyperplane.
All the conditions are satisfied. We conclude that the parameter region of multistationarity is path connected.


In [50]:
#2-site weakly irreversible dephosphorylation, (g)
M,Mp,Mpp,E,ME,MpE,F,MppF,MpF,MpFT,MF = var('M,Mp,Mpp,E,ME,MpE,F,MppF,MpF,MpFT,MF')
species = [M,Mp,Mpp,E,ME,MpE,F,MppF,MpF,MpFT,MF]
reactions = [[M+E,ME],
             [ME,M + E],
             [ME,Mp + E],
             
             [Mp + E,MpE],
             [MpE,Mp + E],
             [MpE,Mpp + E],
             
             [Mpp + F,MppF],
             [MppF,Mpp +F],
             [MppF,MpF],
             [MpF,Mp+F],
             [Mp+F,MpF],
            
             [Mp + F,MpFT],
             [MpFT,Mp+F],
             [MpFT,MF],
             [MF,M+F],
             [M+F,MF]]

F = CheckConnectivity(species,reactions)

n = 11
r = 16
The reaction network is conservative.
The network after removal of intermediates and catalysts is monomolecular.
l = 8
# of positive coefficients: 2176
# of negative coefficients: 640
The support set has a strict separating hyperplane.
All the conditions are satisfied. We conclude that the parameter region of multistationarity is path connected.


In [51]:
#2-site weakly irreversible dephosphorylation, (g) - reduced
M,Mp,Mpp,E,ME,MpE,F,MppF,MpF,MpFT,MF = var('M,Mp,Mpp,E,ME,MpE,F,MppF,MpF,MpFT,MF')
species = [M,Mp,Mpp,E,ME,MpE,F,MppF,MpF,MpFT,MF]
reactions = [[M+E,ME],
             #[ME,M + E],
             [ME,Mp + E],
             
             [Mp + E,MpE],
             #[MpE,Mp + E],
             [MpE,Mpp + E],
             
             [Mpp + F,MppF],
             #[MppF,Mpp +F],
             [MppF,MpF],
             [MpF,Mp+F],
             [Mp+F,MpF],
            
             [Mp + F,MpFT],
             #[MpFT,Mp+F],
             [MpFT,MF],
             [MF,M+F],
             [M+F,MF]]

F= CheckConnectivity(species,reactions)

n = 11
r = 12
The reaction network is conservative.
The network after removal of intermediates and catalysts is monomolecular.
l = 4
# of positive coefficients: 136
# of negative coefficients: 40
The support set has a strict separating hyperplane.
All the conditions are satisfied. We conclude that the parameter region of multistationarity is path connected.


In [52]:
#2-site: fully weakly irreversible, (h)
E,F,S0,S1,S2,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8 = var('E,F,S0,S1,S2,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8')
species = [E,F,S0,S1,S2,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8]
reactions = [[S0 + E,Y1],
                   [Y1,S0 + E],
                   [Y1,Y2],
                   [Y2,S1+E],
                   [S1+E,Y2],
                   
                   
                   [S1 + E,Y3],
                   [Y3,S1 + E],
                   [Y3,Y4],
                   [Y4,S2+E],
                   [S2+E,Y4],
                   
                   [S2 + F,Y5],
                   [Y5,S2 + F],
                   [Y5,Y6],
                   [Y6,S1+F],
                   [S1+F,Y6],
                   
                   [S1 + F,Y7],
                   [Y7,S1 + F],
                   [Y7,Y8],
                   [Y8,S0+F],
                   [S0+F,Y8]
                   ]

F = CheckConnectivity(species,reactions)

n = 13
r = 20
The reaction network is conservative.
The network after removal of intermediates and catalysts is monomolecular.
l = 10
# of positive coefficients: 16320
# of negative coefficients: 3648
The support set does not have a strict separating hyperplane.
The algorithm is inconclusive.


In [53]:
#2-site: fully weakly irreversible, (h) - reduced
E,F,S0,S1,S2,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8 = var('E,F,S0,S1,S2,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8')
species = [E,F,S0,S1,S2,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8]
reactions = [[S0 + E,Y1],
                   #[Y1,S0 + E],
                   [Y1,Y2],
                   [Y2,S1+E],
                   [S1+E,Y2],
                   
                   
                   [S1 + E,Y3],
                   #[Y3,S1 + E],
                   [Y3,Y4],
                   [Y4,S2+E],
                   [S2+E,Y4],
                   
                   [S2 + F,Y5],
                   #[Y5,S2 + F],
                   [Y5,Y6],
                   [Y6,S1+F],
                   [S1+F,Y6],
                   
                   [S1 + F,Y7],
                   #[Y7,S1 + F],
                   [Y7,Y8],
                   [Y8,S0+F],
                   [S0+F,Y8]
                   ]

import time

Start_Time = time.time()
print('Start time: ' + str(Start_Time))
F = CheckConnectivity(species,reactions)

Execution_Time = time.time() - Start_Time
print('Total time: '+str(Execution_Time))

Start time: 1663782327.983872
n = 13
r = 16
The reaction network is conservative.
The network after removal of intermediates and catalysts is monomolecular.
l = 6
# of positive coefficients: 1020
# of negative coefficients: 228
The support set does not have a strict separating hyperplane.
The algorithm is inconclusive.
Total time: 95.37696313858032


In [54]:
#Two-layer cascade with shared kinase, (i)
S0,S1,P0,P1,E,F1,F2,S0E,S1F1,P0S1,P1F2,P0E = var('S0,S1,P0,P1,E,F1,F2,S0E,S1F1,P0S1,P1F2,P0E')
species = [S0,S1,P0,P1,E,F1,F2,S0E,S1F1,P0S1,P1F2,P0E]
reactions = [[S0E, S1+ E],
             [S0 + E,S0E],
             [S0E, S0 +E],
            
            
            [S1F1,S0 + F1],
            [S1 + F1,S1F1],
            [S1F1,S1 + F1],
             
            [P0S1, P1 + S1],
            [P0 + S1, P0S1],
            [P0S1,P0 + S1],
             
            [P0E,P1 + E],
            [P0 + E,P0E],
            [P0E,P0 + E],
             
            [P1F2, P0 + F2],
            [P1 + F2,P1F2],
            [P1F2,P1 + F2]]
import time

Start_Time = time.time()
print('Start time: ' + str(Start_Time))
F = CheckConnectivity(species,reactions,Timing = True)

Execution_Time = time.time() - Start_Time
print('Total time: '+str(Execution_Time))

Start time: 1663782423.398401
n = 12
r = 15
The reaction network is conservative.
The network does not have relevant boundary steady states.
Checking technical conditions: 1663782423.5343542
l = 8
Computing flux cone: 1663782423.5657635
Computing the determinant: 1663782517.5297294
# of positive coefficients: 5088
# of negative coefficients: 224
The support set has a strict separating hyperplane.
All the conditions are satisfied. We conclude that the parameter region of multistationarity is path connected.
Finding separating hyperplane: 1663782547.6258533
Total time: 124.23264074325562


In [55]:
#Two-layer cascade with shared kinase, (i) - reduced
S0,S1,P0,P1,E,F1,F2,S0E,S1F1,P0S1,P1F2,P0E = var('S0,S1,P0,P1,E,F1,F2,S0E,S1F1,P0S1,P1F2,P0E')
species = [S0,S1,P0,P1,E,F1,F2,S0E,S1F1,P0S1,P1F2,P0E]
reactions = [[S0E, S1+ E],
             [S0 + E,S0E],

            [S1F1,S0 + F1],
            [S1 + F1,S1F1],

            [P0S1, P1 + S1],
            [P0 + S1, P0S1],
             
            [P0E,P1 + E],
            [P0 + E,P0E],
             
            [P1F2, P0 + F2],
            [P1 + F2,P1F2]
            ]
import time

Start_Time = time.time()
print('Start time: ' + str(Start_Time))
F = CheckConnectivity(species,reactions)

Execution_Time = time.time() - Start_Time
print('Total time: '+str(Execution_Time))

Start time: 1663782547.6617866
n = 12
r = 10
The reaction network is conservative.
The network does not have relevant boundary steady states.
l = 3
# of positive coefficients: 195
# of negative coefficients: 9
The support set has a strict separating hyperplane.
All the conditions are satisfied. We conclude that the parameter region of multistationarity is path connected.
Total time: 5.907439231872559


In [56]:
#Two layer MAPK cascade, (j)
A,Ap,E,AE,F1,ApF1,  B,Bp,Bpp,F2,BAp,BpAp,BppF2,BpF2 = var('A,Ap,E,AE,F1,ApF1,  B,Bp,Bpp,F2,BAp,BpAp,BppF2,BpF2')
species = [A,Ap,E,AE,F1,ApF1,  B,Bp,Bpp,F2,BAp,BpAp,BppF2,BpF2]
reactions = [[A + E,AE],
             [AE,A+E],
             [AE,Ap + E],
             
             [Ap + F1,ApF1],
             [ApF1,Ap+F1],
             [ApF1,A + F1],
             
             [B + Ap,BAp],
             [BAp,B+Ap],
             [BAp,Bp + Ap],
             [Bp + Ap,BpAp],
             [BpAp,Bp+Ap],
             [BpAp,Bpp + Ap],
             
             [Bpp + F2,BppF2],
             [BppF2,Bpp+F2],
             [BppF2,Bp + F2],
             [Bp + F2,BpF2],
             [BpF2,Bp+F2],
             [BpF2,B + F2]
            ]

F = CheckConnectivity(species,reactions)

n = 14
r = 18
The reaction network is conservative.
The network does not have relevant boundary steady states.
l = 9
# of positive coefficients: 5120
# of negative coefficients: 1408
The support set has a strict separating hyperplane.
All the conditions are satisfied. We conclude that the parameter region of multistationarity is path connected.


In [57]:
#Two layer MAPK cascade, (j) - reduced
A,Ap,E,AE,F1,ApF1,  B,Bp,Bpp,F2,BAp,BpAp,BppF2,BpF2 = var('A,Ap,E,AE,F1,ApF1,  B,Bp,Bpp,F2,BAp,BpAp,BppF2,BpF2')
species = [A,Ap,E,AE,F1,ApF1,  B,Bp,Bpp,F2,BAp,BpAp,BppF2,BpF2]
reactions = [[A + E,AE],
             [AE,Ap + E],
             
             [Ap + F1,ApF1],
             [ApF1,A + F1],
             
             [B + Ap,BAp],
             [BAp,Bp + Ap],
             [Bp + Ap,BpAp],
             [BpAp,Bpp + Ap],
             
             [Bpp + F2,BppF2],
             [BppF2,Bp + F2],
             [Bp + F2,BpF2],
             [BpF2,B + F2]
            ]

F = CheckConnectivity(species,reactions)

n = 14
r = 12
The reaction network is conservative.
The network does not have relevant boundary steady states.
l = 3
# of positive coefficients: 80
# of negative coefficients: 22
The support set has a strict separating hyperplane.
All the conditions are satisfied. We conclude that the parameter region of multistationarity is path connected.


In [58]:
#ERK network
S00,S01,S10,S11,E,F,S00E,S01E,S10E ,S11F,S10F,S01F= var('S00,S01,S10,S11,E,F,S00E,S01E,S10E,S11F,S10F,S01F')
species = [S00,S01,S10,S11,E,F,S00E,S01E,S10E,S11F,S10F,S01F]
reactions = [[S00+E,S00E],
                  [S00E,S00+E],
                  [S00E,S01E],
                  [S01E,S11+E],
                  
                  [S01+E,S01E],
                  [S01E,S01+E],
                  
                  [S10E,S11+E],
                  [S10E,S10+E],
                  [S10+E,S10E],
                  
                  [S11+F,S11F],
                  [S11F,S11+F],
                  [S11F,S10F],
                  [S10F,S00+F],
                  
                  [S10+F,S10F],
                  [S10F,S10+F],
                  
                  [S01F,S00+F],
                  [S01F,S01+F],
                  [S01+F,S01F]]

F = CheckConnectivity(species,reactions)

n = 12
r = 18
The reaction network is conservative.
The network after removal of intermediates and catalysts is monomolecular.
l = 9
# of positive coefficients: 15040
# of negative coefficients: 3432
The support set has a strict separating hyperplane.
All the conditions are satisfied. We conclude that the parameter region of multistationarity is path connected.


In [59]:
#ERK network - reduced
S00,S01,S10,S11,E,F,S00E,S01E,S10E ,S11F,S10F,S01F= var('S00,S01,S10,S11,E,F,S00E,S01E,S10E,S11F,S10F,S01F')
species = [S00,S01,S10,S11,E,F,S00E,S01E,S10E,S11F,S10F,S01F]
reactions = [[S00+E,S00E],
                  [S00E,S01E],
                  [S01E,S11+E],
                  
                  [S01+E,S01E],
                  [S01E,S01+E],
                  
                  [S10E,S11+E],
                  [S10+E,S10E],
                  
                  [S11+F,S11F],
                  [S11F,S10F],
                  [S10F,S00+F],
                  
                  [S10+F,S10F],
                  [S10F,S10+F],
                  
                  [S01F,S00+F],
                  [S01+F,S01F]]

F = CheckConnectivity(species,reactions)
                

n = 12
r = 14
The reaction network is conservative.
The network after removal of intermediates and catalysts is monomolecular.
l = 5
# of positive coefficients: 1374
# of negative coefficients: 340
The support set has a strict separating hyperplane.
All the conditions are satisfied. We conclude that the parameter region of multistationarity is path connected.
